In [7]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import re
import numpy as np
from scipy.optimize import curve_fit

In [15]:
def thermal_decay(t, t_decay, temp_initial, temp_final):
    return (temp_initial-temp_final) * np.exp(-t/t_decay) + temp_final

In [20]:
thermal_decay(14, 1 , 50, 25)

25.000020788217977

In [1]:
wait_time = 0.08 # seconds

In [21]:
def board_io(  command, 
             output = 0, # no. of characters to read
             wait_time = wait_time # waiting time after writing a command before-->
             #reading the output to make sure that the board has enough time to respond
            ):
    
    bytes_to_write = bytes( command + '\r', encoding = 'utf-8')
    n_bytes_written = smartboard.write( bytes_to_write )
    smartboard.flush()
    
    time.sleep(wait_time)
    
    if output == 0:
        output = smartboard.in_waiting
        
    bytes_read = smartboard.read(output)
        
    return bytes_read.decode().split('\r\n')[:-1]

In [4]:
# Turns off the laser (gc=0) scans the bench2 temperature and measures comb and fscs resistances
# bench2 is disabled after the scan
def scan_bench_for_comb_and_fscs ( 
                                temp_min = 25, temp_max = 35, temp_step = 0.1,
                                bench_wait_time = 1, # wait time after setting the bench2 target temperature et each scan
                                k_bench=[] # default pid parameters
                               ):
    
    if len(k_bench) == 0: # if user has not defined it
        k_bench = get_pid(heater ='bench2')
        print('PID for bench: ' + str(k_bench))
    
    # Turn off the GC
    gc_read = board_io( 'gc 0')[-1]
    gc_read = float( re.findall('[0-9-.]+' , gc_read)[0] ) # extract the current from the returned string
    print('Gain Current is %.3f mA' %gc_read)
    
    # Set Target Temperature and PID of bench
    target_c = temp_min #Celcius 

    board_io( 'bench2 target %s' %target_c) # Set Target T of bench
    board_io( 'bench2 pid %s %s %s' %( tuple(k_bench) )) # Set PID of bench
    board_io( 'bench2 enable on')# Enable bench
    
    # Read the Temperatue of bench2 to make sure it is at the target temperature
    temp_actual = board_io( 'bench2 actual')[-1]
    temp_actual = float ( re.findall('[0-9-.]+' , temp_actual)[0]) # extract the temperature from the returned string
    
    # Disable the comb and fscs heater closed loops as they are not in use, but being measured
    board_io( 'fscs enable off'); board_io( 'fscs heat 0')
    board_io( 'comb enable off'); board_io( 'comb heat 0')
    
    temp_range = range(temp_min, temp_max + temp_step, temp_step)
    temp = []; temp_diff = []; rtd_r = []; fscs = []; temp_comb =[] ; temp_fscs = []
    
    user_input = 0; i=0;

    start_time = time.time()
    
    while( user_input != '1' ):
        print( '%.3f °C (target T: %.3f °C) ΔT=%.2f C' %(temp_actual, target_c, abs(target_c-temp_actual)))
        # Get confirmation from the user that the actual temperature is good enough to contiune
        user_input = input("Okay to continue ?: (0 or 1) ")
        i+=1 # allows only 10 steps in this while loop
        if i > 10:
            break
        
        temp_actual = board_io( 'bench2 actual')[-1]
        temp_actual = float ( re.findall('[0-9-.]+' , temp_actual)[0]) # extract the temperature from the returned string
    
    print('Starting measurement')
    # scan over the bench2 target temperatures and perform measurements
    for temp_write in temp_range:

        board_io( 'bench2 target %s' %temp_write)#
        time.sleep(bench_wait_time) # so that temperature stabilizes 

        # Measure bench2 Temperature
        temp_actual = board_io( 'bench2 actual')[-1]
        temp_actual = float ( re.findall('[0-9-.]+' , temp_actual)[0]) # extract the temperature from the returned string
        temp.append(temp_actual)
        temp_diff.append( abs(temp_write-temp_actual) ) # difference between the input and the read temp value
        
        # Measure Comb (RTD) Temperature
        temp_comb_read = board_io( 'comb actual')[-1]
        temp_comb_read = float ( re.findall('[0-9-.]+' , temp_comb_read)[0]) # extract the comb temperature from the returned string
        temp_comb.append(temp_comb_read)
        
        # Measure RTD Resistance
        rtd_r_read = board_io( 'comb rtd_r')[-1]
        rtd_r_read = float ( re.findall('[0-9-.]+' , rtd_r_read)[0]) # extract the resistance from the returned string
        rtd_r.append(rtd_r_read)
        
        # Measure FSCS Temperature
        temp_fscs_read = board_io( 'fscs actual')[-1]
        temp_fscs_read = float ( re.findall('[0-9-.]+' , temp_fscs_read)[0]) # extract the comb temperature from the returned string
        temp_fscs.append(temp_fscs_read)
        
        # Measure FSCS Thermistor Resistance
        fscs_read = board_io( 'fscs thermistor_r')[-1]
        fscs_read = float ( re.findall('[0-9-.]+' , fscs_read)[0]) # extract the resistance from the returned string
        fscs.append(fscs_read)

        print( '%.3f °C (target T: %.3f °C) ΔT=%.2f C' %(temp_actual, temp_write, abs(temp_write-temp_actual)))

    print('Max difference between target & actual T = %.3f °C' %max(temp_diff))

    # Turn off the bench
    board_io( 'bench2 enable off')
    
    elapsed_time = time.time() - start_time
    print('Scanning took %.1f seconds' %elapsed_time )
                           
    # Plot the RTD - Temperature curve
    theme = theme_pyplot()
    data_title = 'bench2 at %.1f °C, PID: %.2f, %.2f, %.2f, wait time: %d s' %(target_c, k_bench[0], k_bench[1], k_bench[2], bench_wait_time)
    temp_axis_label = 'Temperature (°C)'; 
    rtd_r_axis_label = 'RTD (ohm)'; fscs_r_axis_label = 'FSCS Thermistor (ohm)'
    comb_T_axis_label = 'COMB Temperature (°C)'; fscs_T_axis_label = 'FSCS Temperature (°C)'

  
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize = (theme.figsize[0], theme.figsize[1]*2 ))
    plt.rcParams['axes.linewidth'] = 3 #set the value globally
    
    p = np.polyfit(temp, rtd_r,  1) # linear fit of RTD resistance

    R_RT = p[1] + p[0]*25 # RTD resistance at 25 °C
    TCR = p[0]/R_RT # RTD resistance temperature coefficient
    print('RTD(T) = %.6f(1 + %.6f(T - 25 °C) ohms ) [Fit Result]' %(R_RT, TCR) )
    
    TCR_firmware = float( re.findall('[0-9-.]+' , board_io( 'comb tcr')[-1])[0] ); R_RT_firmware = float( re.findall('[0-9-.]+' , board_io( 'comb r_rt')[-1])[0] )
    print('RTD(T) = %.6f(1 + %.6f(T - 25 °C) ohms ) [Firmware]' %(R_RT_firmware, TCR_firmware) )

    
    theme.line_kwargs['label'] = 'Measurement'
    ax1.plot(temp, rtd_r, **theme.line_kwargs)
    theme.line_kwargs['ls'] = '--'; theme.line_kwargs['marker'] = ''; theme.line_kwargs['label'] = 'Linear Fit'
    theme.line_kwargs['color'] = 'red'
    ax1.plot(temp, p[1] + np.asarray(temp)*p[0], **theme.line_kwargs) # linear fit of RTD resistance
    
    ax1.set_xlabel(temp_axis_label, fontsize = theme.fontsize); ax1.set_ylabel(rtd_r_axis_label, fontsize = theme.fontsize)
    ax1.grid(**theme.grid_kwargs); ax1.tick_params(**theme.tick_kwargs); 
    ax1.legend(**theme.legend_kwargs)
    
    theme.line_kwargs['label'] = 'FSCS Thermistor Resistance (ohms)'; theme.line_kwargs['ls'] = '-';
    ax2.plot(temp, fscs, **theme.line_kwargs)
    ax2.update_from(ax1)
    ax2.set_xlabel(temp_axis_label, fontsize = theme.fontsize); ax2.set_ylabel(fscs_r_axis_label, fontsize = theme.fontsize)
    ax2.grid(**theme.grid_kwargs); ax2.tick_params(**theme.tick_kwargs)
    
    theme.line_kwargs['label'] = ''; 
    ax3.plot(temp, temp_comb, **theme.line_kwargs)
    theme.line_kwargs['color'] = 'blue'; theme.line_kwargs['ls'] = '--';
    ax3.plot(temp, temp, **theme.line_kwargs)
    ax3.set_xlabel(temp_axis_label, fontsize = theme.fontsize); ax3.set_ylabel(comb_T_axis_label, fontsize = theme.fontsize)
    ax3.grid(**theme.grid_kwargs); ax3.tick_params(**theme.tick_kwargs)
    
    theme.line_kwargs['label'] = ''; theme.line_kwargs['ls'] = '-';
    ax4.plot(temp, temp_fscs, **theme.line_kwargs)
    theme.line_kwargs['color'] = 'blue'; theme.line_kwargs['ls'] = '--';
    ax4.plot(temp, temp, **theme.line_kwargs)
    ax4.set_xlabel(temp_axis_label, fontsize = theme.fontsize); ax4.set_ylabel(fscs_T_axis_label, fontsize = theme.fontsize)
    ax4.grid(**theme.grid_kwargs); ax4.tick_params(**theme.tick_kwargs)
    
    plt.tight_layout()
    plt.show()
    
    return temp, rtd_r, fscs, R_RT, TCR

In [2]:
def scan_comb_fscs_open_loop( 
                            comb_heat_min = 0, comb_heat_max = 15, comb_heat_step = 1,
                            fscs_heat_min = 0, fscs_heat_max = 15, fscs_heat_step = 1,
                             target_c = 25,
                        bench_wait_time = 5 # wait time after setting the bench2 target temperature et each scan
                           ):

    # Turn off the GC
    gc_read = board_io( 'gc 0')[-1]
    gc_read = float( re.findall('[0-9-.]+' , gc_read)[0] ) # extract the current from the returned string
    print('Gain Current is %.3f mA' %gc_read)
    
    # Set Target Temperature and PID of bench
    
    k_bench = get_pid(heater ='bench2')
    board_io( 'bench2 target %s' %target_c) # Set Target T of bench
    board_io( 'bench2 pid %s %s %s' %( tuple(k_bench) )) # Set PID of bench
    board_io( 'bench2 enable on')# Enable bench
                       
    # Disable the comb and fscs closed loop heaters as they are not in use, open loop is in use
    board_io( 'fscs enable off'); board_io( 'fscs heat 0')
    board_io( 'comb enable off'); board_io( 'comb heat 0')
    
    # Read the Temperatue of bench2 to make sure it is at the target temperature
    temp_actual = board_io( 'bench2 actual')[-1]
    temp_actual = float ( re.findall('[0-9-.]+' , temp_actual)[0]) # extract the temperature from the returned string
   
    comb_heat_range = range(comb_heat_min, comb_heat_max + comb_heat_step, comb_heat_step)
    fscs_heat_range = range(fscs_heat_min, fscs_heat_max + fscs_heat_step, fscs_heat_step)

    temp_comb = []; rtd_r = []; temp_fscs = []; i_tecb = []; power_tecb = []

    user_input = 0; i=0;
                       
    while( user_input != '1'):
        print( '%.3f °C (target T: %.3f °C) ΔT=%.2f C' %(temp_actual, target_c, abs(target_c-temp_actual)))
        # Get confirmation from the user that the actual temperature is good enough to contiune
        user_input = input("Okay to continue ?: (0 or 1) ")
        i+=1 # allows only 10 steps in this while loop
        if i > 10:
            break
        
        temp_actual = board_io( 'bench2 actual')[-1]
        temp_actual = float ( re.findall('[0-9-.]+' , temp_actual)[0]) # extract the temperature from the returned string
                           
    start_time = time.time()
    print('Starting measurement')
    for comb_heat_write in comb_heat_range:

        board_io( 'comb heat %s' %comb_heat_write) # give heat to the comb

        time.sleep(bench_wait_time) # so that comb temperature stabilizes 

        # Measure Comb (RTD) Temperature
        temp_comb_read = board_io( 'comb actual')[-1]
        temp_comb_read = float ( re.findall('[0-9-.]+' , temp_comb_read)[0]) # extract the comb temperature from the returned string
        temp_comb.append(temp_comb_read)
        
        
    for fscs_heat_write in fscs_heat_range:

        board_io( 'fscs heat %s' %fscs_heat_write) # give heat to the comb

        time.sleep(bench_wait_time) # so that fscs temperature stabilizes 

        # Measure FSCS Temperature
        temp_fscs_read = board_io( 'fscs actual')[-1]
        temp_fscs_read = float ( re.findall('[0-9-.]+' , temp_fscs_read)[0]) # extract the comb temperature from the returned string
        temp_fscs.append(temp_fscs_read)
        
        # Measure FSCS Current (If the T-E-C-B+, T-E-C-B- pins are used)
        i_tecb_read = board_io( 'adc itecb')[-1]
        i_tecb_read = float ( re.findall('[0-9-.]+' , i_tecb_read)[0])
        i_tecb.append(i_tecb_read)
    
    # FSCS heater resistance
    fscs_heater_r = board_io( 'fscs coarse_r')[-1]
    fscs_heater_r = float ( re.findall('[0-9-.]+' , fscs_heater_r)[0])
    
    power_tecb = [1000*i*i*fscs_heater_r for i in i_tecb] #mW, actual electrical power used on the fscs heater resistor
                            
    # Turn off the bench
    board_io( 'bench2 enable off')[-1]; print('bench2 is off now') 

    # Turn off the comb and fscs open loop heaters
    board_io( 'comb heat 0'); print('Comb heater is off now')
    board_io( 'fscs heat 0'); print('FSCS heater is off now')                   

    elapsed_time = time.time() - start_time
    print('Scanning took %.1f seconds' %elapsed_time )
                            
    # Plot the RTD - COMB power curve
    # Plot the RTD - Temperature curve
    theme = theme_pyplot()
    data_title = 'T = %.1f °C, PID: %.2f, %.2f, %.2f, wait time: %d s' %(target_c, k_bench[0], k_bench[1], k_bench[2], bench_wait_time)
    comb_heat_axis_label = 'COMB Heating Power (mW)'; fscs_heat_axis_label = 'FSCS Heating Power (mW)'
    temp_comb_axis_label = 'COMB Temperature (°C)'; temp_fscs_axis_label = 'FSCS Temperature (°C)'
    power_tecb_axis_label = 'Applied Power (mW)'; 


    fig, ((ax1,ax2), (ax3,ax4) ) = plt.subplots(2, 2, figsize = theme.figsize)
    plt.rcParams['axes.linewidth'] = 3 #set the value globally
    
    ax1.set_xlabel(comb_heat_axis_label, fontsize = theme.fontsize); ax1.set_ylabel(temp_comb_axis_label, fontsize = theme.fontsize)
    ax1.grid(**theme.grid_kwargs); ax1.tick_params(**theme.tick_kwargs); ax1.legend(**theme.legend_kwargs)
    
    ax2.update_from(ax1)
    ax2.set_xlabel(fscs_heat_axis_label, fontsize = theme.fontsize); ax2.set_ylabel(temp_fscs_axis_label, fontsize = theme.fontsize)
    
    ax4.set_xlabel(fscs_heat_axis_label, fontsize = theme.fontsize); ax4.set_ylabel(power_tecb_axis_label, fontsize = theme.fontsize)

    comb_open_loop_fit = np.polyfit(comb_heat_range, temp_comb, 1) # linear fit of comb open loop temperature-heat scan
    print('COMB(T) = %.3f °C + %.3f * Power/mW )' %(comb_open_loop_fit[1], comb_open_loop_fit[0]) )
    
    fscs_open_loop_fit = np.polyfit(fscs_heat_range, temp_fscs, 1) # linear fit of fscs open loop temperature-heat scan
    print('FSCS(T) = %.3f °C + %.3f * Power/mW )' %(fscs_open_loop_fit[1], fscs_open_loop_fit[0]) )

    theme.line_kwargs['label'] = 'Comb Open Loop'
    ax1.plot(comb_heat_range, temp_comb, **theme.line_kwargs)
    ax1.plot(comb_heat_range, comb_open_loop_fit[1] + comb_open_loop_fit[0]*np.asarray(comb_heat_range), **theme.line_kwargs)

    theme.line_kwargs['label'] = 'FSCS Open Loop'
    ax2.plot(fscs_heat_range, temp_fscs, **theme.line_kwargs)
    ax2.plot(fscs_heat_range, fscs_open_loop_fit[1] + fscs_open_loop_fit[0]*np.asarray(fscs_heat_range), **theme.line_kwargs)
    
    ax4.plot(fscs_heat_range, power_tecb, **theme.line_kwargs)

    
    plt.tight_layout()
    plt.show()
    
    return comb_heat_range, temp_comb, fscs_heat_range, temp_fscs

In [3]:
# This function performs a step reponse of the bench2 from an inital temperature to a final temperature
def bench_step_response(temp_initial = 25,
                        temp_final = 27,
                        n_steps = 100, # number of steps to monitor the bench2 temperature
                        k_bench=[]
                       
                       ):

    if len(k_bench) == 0: # if user has not defined it
        k_bench = get_pid(heater ='bench2')
        print('PID for bench: ' + str(k_bench))
        
    # Set Target Temperature and PID of bench
    board_io( 'bench2 target %s' %temp_initial)
    board_io( 'bench2 pid %s %s %s' %( tuple(k_bench) ))

    # Enable bench
    board_io( 'bench2 enable on' )
    
    n_range = range(0, n_steps + 1, 1)

    temp_time = []; time_measured = []
    
    temp_actual = board_io( 'bench2 actual')[-1]
    temp_actual = float ( re.findall('[0-9-.]+' , temp_actual)[0]) # extract the temperature from the returned string
    
    user_input = 0; i=0;
                       
    while( user_input != '1'):
        print( '%.3f °C (target T: %.3f °C) ΔT=%.2f C' %(temp_actual, temp_initial, abs(temp_initial-temp_actual)))
        # Get confirmation from the user that the actual temperature is good enough to contiune
        user_input = input("Okay to continue ?: (0 or 1) ")
        i+=1 # allows only 10 steps in this while loop
        if i > 10:
            break
        
        temp_actual = board_io( 'bench2 actual')[-1]
        temp_actual = float ( re.findall('[0-9-.]+' , temp_actual)[0]) # extract the temperature from the returned string

    # Set to the end temperature
    board_io( 'bench2 target %s' %temp_final)
    start_time = time.time()
    print('Starting measurement')
    for n in n_range:

        time_measured.append( time.time() - start_time )

        temp_actual = board_io( 'bench2 actual')[-1]
        temp_actual = float ( re.findall('[0-9-.]+' , temp_actual)[0]) # extract the temperature from the returned string
        temp_time.append(temp_actual)

    elapsed_time = time.time() - start_time

    # reset the bench
    board_io( 'bench2 target %s' %temp_initial)

    # Plot the Temperature
    data_title = 'Initial = %.1f °C, Final = %.1f °C. PID: %.2f, %.2f, %.2f' %(temp_initial, temp_final, k_bench[0], k_bench[1], k_bench[2])
    x_axis_label = 'Time (s)'; y_axis_label = 'Temperature (°C)'
    
    theme = theme_pyplot()
    fig, ax = plt.subplots(figsize=theme.figsize)
    plt.rcParams['axes.linewidth'] = 3 #set the value globally

    theme.line_kwargs['label'] = data_title
    ax.plot(time_measured, temp_time, **theme.line_kwargs)
    ax.grid(**theme.grid_kwargs); ax.tick_params(**theme.tick_kwargs); ax.legend(**theme.legend_kwargs)

    ax.set_xlabel(x_axis_label, fontsize = theme.fontsize); ax.set_ylabel(y_axis_label, fontsize = theme.fontsize)
    plt.show()
    
    return time_measured, temp_time

In [2]:
# This function monitors stedy-state or free-cooling reponse of any heaters at a set temperature
def heater_steady_response(heater = 'comb',
                      is_heater_on = 1,
                      t_initial = 25,
                           t_final = 0,
                        n_steps = 100, # number of steps to monitor the temperature
                        k_heater=[] # default pid parameters
                       
                       ):
    
    # Set Target Temperature and PID of heater
    if len(k_heater) == 0: # if user has not defined it
        k_heater = get_pid(heater = heater)
        print('PID for ' + heater + ' : ' + str(k_heater))
    if t_final == 0: # if user does not want to change the temperature; steady-state response
        t_final = t_initial
        
    board_io( heater + ' target %s' %t_initial)
    board_io( heater + ' pid %s %s %s' %( tuple(k_heater) ))
    if is_heater_on:
        # Enable bench
        board_io( heater + ' enable on')

    n_range = range(0, n_steps + 1, 1)

    temp_time = []; time_measured = []; temp_sled = []; heat_time=[]
    
    temp_actual = board_io( heater + ' actual')[-1]
    temp_actual = float ( re.findall('[0-9-.]+' , temp_actual)[0]) # extract the temperature from the returned string
    
    user_input = 0; i=0;
                       
    while( user_input != '1'):
        print( '%.3f °C (target T: %.3f °C) ΔT=%.2f C' %(temp_actual, t_initial, abs(t_initial-temp_actual)))
        # Get confirmation from the user that the actual temperature is good enough to contiune
        user_input = input("Okay to continue ?: (0 or 1) ")
        i+=1 # allows only 10 steps in this while loop
        if i > 10:
            break
        
        temp_actual = board_io( heater + ' actual')[-1]
        temp_actual = float ( re.findall('[0-9-.]+' , temp_actual)[0]) # extract the temperature from the returned string
    
    
    board_io( heater + ' target %s' %t_final)
    #if not is_heater_on:
        # Disable bench2 if user wants to monitor cooling
        #board_io( heater + ' enable off')
    
    start_time = time.time()
    print('Starting measurement')
    for n in n_range:

        time_measured.append( time.time() - start_time )

        temp_actual = board_io( heater + ' actual')[-1]
        temp_actual = float ( re.findall('[0-9-.]+' , temp_actual)[0]) # extract the temperature from the returned string
        temp_time.append(temp_actual)
        
        if heater != 'bench2':
            
            #heat_read = board_io( heater + ' heat')[-1]
            #heat_read = board_io('adc rtcase')[-1]
            #heat_read = int(re.findall('(?<=: )[^\]]+' , board_io('mem rl 0xa0000040 1')[-1])[0], 16)
            #heat_read = float ( re.findall('[0-9-.]+' , heat_read)[0]) # extract the temperature from the returned string
            #heat_read = 1/(math.log (heat_read/(4.096-heat_read))/3950 + 1/298.15)-273.15
            heat_read = float (re.findall ( '(?<=\=)(.*?)(?=\,)', board_io('fscs last')[-1]) [1])
            heat_time.append(heat_read)
        elif heater == 'bench2':
            heat_read = board_io( heater + ' control')[-1]
            heat_read = float ( re.findall('[0-9-.]+' , heat_read)[0]) # extract the temperature from the returned string
            heat_time.append(heat_read)
            
        
    elapsed_time = time.time() - start_time
    print('Measurement took %.1f seconds' %elapsed_time )
    board_io( heater + ' target %s' %t_initial)

    squared_error = [(t_final - i)**2 for i in temp_time]
    variance =  sum(squared_error)/len(temp_time)
    std_deviation = variance**0.5
    # Plot the Temperature
    data_title = 'Std Deviation = %.4f °C,Target = %.1f °C. PID: %.2f, %.2f, %.2f' %(std_deviation, t_final, k_heater[0], k_heater[1], k_heater[2])
    x_axis_label = 'Time (s)'; temp_axis_label = 'Temperature (°C)'; 
    
    if heater == 'bench2':
        heat_axis_label = 'Current (mA)'
    else :
        heat_axis_label = 'Heat (mW)'
    
    theme = theme_pyplot()
    theme.figsize = (10,8)
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=theme.figsize)
    plt.rcParams['axes.linewidth'] = 3 #set the value globally

    theme.line_kwargs['label'] = data_title #, theme.line_kwargs['ms'] = 6
    ax1.plot(time_measured, temp_time, **theme.line_kwargs)
    
    
    if not is_heater_on: # if a free cooling is desired, we can fit it
        # exponential fit to the thermal decay
        popt, pcov = curve_fit(thermal_decay, time_measured, temp_time)
        t_decay, temp_initial, temp_final = popt
        ax1.plot(np.asarray(time_measured), thermal_decay(np.asarray(time_measured), *popt), 'r--', label="%.3f*exp(-t/%.3f) + %.3f" %(temp_initial-temp_final, t_decay, temp_final) ) 

    theme.line_kwargs['color'] = 'r'
    ax2.plot(time_measured, heat_time, **theme.line_kwargs)
    
    theme.legend_kwargs['fontsize'] = theme.fontsize/2
    ax1.grid(**theme.grid_kwargs); ax1.tick_params(**theme.tick_kwargs); ax1.legend(**theme.legend_kwargs, title=heater)
    ax2.grid(**theme.grid_kwargs); ax2.tick_params(**theme.tick_kwargs); ax2.legend(**theme.legend_kwargs, title=heater)
    

    ax1.set_xlabel(x_axis_label, fontsize = theme.fontsize); ax1.set_ylabel(temp_axis_label, fontsize = theme.fontsize)
    ax2.set_xlabel(x_axis_label, fontsize = theme.fontsize); ax2.set_ylabel(heat_axis_label, fontsize = theme.fontsize)
    plt.tight_layout()
    plt.show()
    
    return time_measured, temp_time, heat_time

# Bias Temperature, Scan Gain Chip Current

In [8]:
def bias_T_scan_gc(data_title='',
                   gc_min = 49 , gc_max = 50, gc_step = -1, # Set min, max gain current, the step size in mA
                   temp_min = 22, temp_max = 30, temp_step = -1, # Set min, max temperature, the step size in Celcius
                   k_bench= [], 
                   bench_enabled = 'on', # enable bench
                   temp_fscs = 25, k_fscs = [], fscs_enabled = 'on', # fscs temperature and PID
                   temp_comb = 25, k_comb = [], comb_enabled = 'on', # comb temperature and PID
                   bench_wait_time = 3, # wait time between each step while scanning the temperature so that the loop stabilizes
                   gc_wait_time = 0.5, # wait time at each gc step during the scan
                   mode_lock = 1  
                   
):
    
    if data_title == '':
        raise ValueError('Always provide data_title')
        
    start_time = time.time()

    # Prepare the Temperature and Gain Current Ranges
    temp_range = np.arange(temp_max, temp_min + temp_step, temp_step) if temp_step < 0 else np.arange(temp_min, temp_max + temp_step, temp_step)    
    gc_range = np.arange(gc_max, gc_min + gc_step, gc_step) if gc_step < 0 else np.arange(gc_min, gc_max + gc_step, gc_step) 

    temp_bench2 = temp_range[0] # target is the first temperature value
    
    if not mode_lock:
        
        # Initialize the bench
        if len(k_bench) == 0: # if user has not defined it
            k_bench = get_pid(heater ='bench2')
            print('PID for bench: ' + str(k_bench))
            
        board_io( 'bench2 target %s' %temp_bench2) 
        board_io( 'bench2 pid %s %s %s' %(tuple(k_bench))) # set the PID
        board_io( 'bench2 enable '+ bench_enabled)     # Enable bench

        # Initialize the FSCS Heater
        if len(k_fscs) == 0: # if user has not defined it
            k_fscs = get_pid(heater ='fscs')
            print('PID for fscs: ' + str(k_fscs))
            
        board_io( 'fscs target %s' %temp_fscs) 
        board_io( 'fscs pid %s %s %s' %(tuple(k_fscs))) # set the PID
        board_io( 'fscs enable ' + fscs_enabled) # Enable the heater

        # Initialize the Comb Heater
        if len(k_comb) == 0: # if user has not defined it
            k_comb = get_pid(heater ='comb')
            print('PID for comb: ' + str(k_comb))
            
        board_io( 'comb target %s' %temp_comb) 
        board_io( 'comb pid %s %s %s' %(tuple(k_comb))) # set the PID
        board_io( 'comb enable ' + comb_enabled) # Enable the heater

    
    time_measured = []
    mpd_temp = []; power_temp = []; temp_diff = []; frequency_temp = []; gc = []; mod_i_oop_temp = []; mod_q_oop_temp= []
    gc_diff = []; frequency_read = 0 ; temp = []; 
    fscs_heat_temp = []; comb_heat_temp = []
    T_fscs_actual_temp = []; T_bench_actual_temp = []; T_comb_actual_temp = [] 


    power_meter.sense.power.dc.range.upper = 60/1000 # 60 mW, set to maximum range of optical power by PM100D

    mpd_temp = []; power_temp = []; temp = []; temp_diff = []; frequency_temp = []

    power_meter.sense.power.dc.range.upper = 60/1000 # 60 mW, set to maximum range of optical power by PM100D

    for temp_write in temp_range:

        
        if mode_lock:
            board_io( 'bench2 mode oop')
            print_thermal_state()
        
        if not mode_lock:
            
            # change to the first value of gc_range and change temperature later and wait
            # so that this abrupt change will not cause any abnomalities
            gc_read = board_io( 'gc %s' %gc_range[0])#
        
        
            # Switch back to thermal control on the bench
            board_io( 'bench2 mode temp') 
            board_io( 'bench2 target %s' %temp_write)#    
            time.sleep(bench_wait_time) # so that temperature stabilizes 

        temp_actual = board_io( 'bench2 actual')[-1]
        temp_actual = float ( re.findall('[0-9-.]+' , temp_actual)[0]) # extract the temperature from the returned string

        temp.append(temp_actual)
        temp_diff.append( abs(temp_write - temp_actual) ) # difference between the input and the read temp value

        print('T = %.2f °C (Target: %.2f °C)' %(temp_actual, temp_write) )

        gc = []; gc_diff = []; power = []; mpd = []; frequency = []
        power = []; mpd = []; frequency = []; mod_i_oop = []; mod_q_oop = []; 
        fscs_heat=[]; comb_heat=[]
        T_fscs_actual = []; T_bench_actual = []; T_comb_actual = [] 
        

        for gc_write in gc_range:

            #if not mode_lock:

            #    gc_read = board_io( 'gc %s' %gc_write)[-1]#
                
            #if  mode_lock:
            #    gc_read = board_io( 'gc')[-1]#
            
            gc_read = board_io( 'gc %s' %gc_write)[-1]#

            print(gc_read)
            gc_read = float( re.findall('[0-9-.]+' , gc_read)[0] ) # extract the current from the returned string
            gc_diff.append( abs(gc_write-gc_read) ) # difference between the input and the read gc value
            gc.append(gc_read)

            time.sleep(gc_wait_time) # so that the spectrum stabilizes 
            
            time_measured.append( time.strftime("%Y-%m-%d_%H:%M:%S") )

            # FSCS Actual Temperature
            T_fscs_actual_read = board_io( 'fscs actual')[-1]
            T_fscs_actual_read = float ( re.findall('[0-9-.]+' , T_fscs_actual_read)[0]) # extract the temperature from the returned string
            T_fscs_actual.append(T_fscs_actual_read)

            # bench2 Actual Temperature
            T_bench_actual_read = board_io( 'bench2 actual')[-1]
            T_bench_actual_read = float ( re.findall('[0-9-.]+' , T_bench_actual_read)[0]) # extract the temperature from the returned string
            T_bench_actual.append(T_bench_actual_read)
            
            temp_diff.append( abs(temp_write - T_bench_actual_read) ) # difference between the input and the read temp value
            

            # COMB Actual Temperature
            T_comb_actual_read = board_io( 'comb actual')[-1]
            T_comb_actual_read = float ( re.findall('[0-9-.]+' , T_comb_actual_read)[0]) # extract the temperature from the returned string
            T_comb_actual.append(T_comb_actual_read)

            temp_diff.append( abs(temp_write - T_bench_actual_read) ) # difference between the input and the read temp value

            ### Measurements ###
            # Optical Power from Thorlabs PM100D
            power_read = power_meter.read * 1000 # converted to mW
            if power_read > 1000 : # if the PM100D is saturated
                power_read = 0

            power.append(power_read)

            # Optical Power and Peak frequency from Bristol 438A Wavelength Meter
            #power_bristol.append(float(bristol.getSimpleMsg(b':MEAS:SCAL:ALL?').split(', ')[3]))
            frequency_read = float(bristol.getSimpleMsg(b':MEAS:SCAL:ALL?').split(', ')[2])
            frequency.append(frequency_read)

            # Read oop, connected to mpd photodiode on TV0, adc to read may be 'oop' or 'ap'
            mpd_read = board_io( 'adc oop')[-1]
            mpd_read = float (re.findall('[0-9-.]+', mpd_read)[0]) # extract the mpd value from the returned string
            mpd.append(mpd_read) 

            # Read 'mod i oop' and 'mod q oop'
            mod_iq_read = board_io( 'mod iq oop')[-1]
            mod_i_oop_read = float(re.findall('[0-9-.]+', mod_iq_read)[0]); mod_q_oop_read = float(re.findall('[0-9-.]+', mod_iq_read)[-1])
            mod_i_oop.append(mod_i_oop_read); mod_q_oop.append(mod_q_oop_read); 
            
            fscs_heat_read = board_io( 'fscs heat')[-1]
            fscs_heat_read = float(re.findall('[0-9-.]+', fscs_heat_read)[0])
            fscs_heat.append(fscs_heat_read)

            comb_heat_read = board_io( 'comb heat')[-1]
            comb_heat_read = float(re.findall('[0-9-.]+', comb_heat_read)[0])
            comb_heat.append(comb_heat_read)

            print( '%.2f °C (actual %.2f °C), Comb:%.2f °C, FSCS:%.2f °C, %.1f mA P.current, %.1f mW Opt.Power, %.5f THz, %.1f mA GC' %(temp_write,T_bench_actual_read, T_comb_actual_read, T_fscs_actual_read, mpd_read, power_read, frequency_read, gc_read))
            
            if T_bench_actual_read <21 or T_bench_actual_read > 27:
                print('Mode-locking did not work')
                print_thermal_state()
                break
            

        mod_i_oop_temp.append(mod_i_oop); mod_q_oop_temp.append(mod_q_oop); 
        mpd_temp.append(mpd); power_temp.append(power); frequency_temp.append(frequency)
        fscs_heat_temp.append(fscs_heat); comb_heat_temp.append(comb_heat)
        T_fscs_actual_temp.append(T_fscs_actual); T_bench_actual_temp.append(T_bench_actual); T_comb_actual_temp.append(T_comb_actual)


        elapsed_time = time.time() - start_time
        print('Elapsed time %.1f seconds (= %.2f hours)' %(elapsed_time, elapsed_time/3600) )
        

        
    print('Max difference between target & actual T = %.3f °C' %max(temp_diff))
    
    if not mode_lock:
        # Switch back to thermal control on the bench
        board_io( 'bench2 mode temp') 

        # Set the bench2 back to temp_min
        board_io( 'bench2 target %s' %min(temp_range))
        print('bench2 is set to %s' %min(temp_range))

        # Turn off the GC
        gc_read = board_io( 'gc 0')[-1]
        gc_read = float( re.findall('[0-9-.]+' , gc_read)[0] ) # extract the current from the returned string
    
    print('Gain Current is %.3f mA now' %gc_read)
    elapsed_time = time.time() - start_time
    print('All took %.1f seconds (= %.2f hours)' %(elapsed_time, elapsed_time/3600) )

    # No need to Transpose the measured arrays because the scan direction is Bias T, Scan GC

    return [time_measured, temp_range, gc_range, mpd_temp, power_temp, frequency_temp, mod_i_oop_temp, mod_q_oop_temp, T_fscs_actual_temp, T_bench_actual_temp, T_comb_actual_temp, fscs_heat_temp, comb_heat_temp]

# Bias Gain Chip Current, Scan Temperature

In [10]:
def bias_gc_scan_T(data_title='',
                   gc_min = 49 , gc_max = 50, gc_step = -1, # Set min, max gain current, the step size in mA
                   temp_min = 22, temp_max = 30, temp_step = -1, # Set min, max temperature, the step size in Celcius
                   heater = 'bench2', # the heater to scan T
                   temp_bench2 = 25, k_bench = [], bench_enabled = 'on', # bench2 temperature & PID
                   temp_fscs  = 25, k_fscs =  [], fscs_enabled = 'on', # fscs temperature & PID
                   temp_comb  = 25, k_comb =  [], comb_enabled = 'on', # comb temperature & PID
                   
                   bench_wait_time = 3, # wait time between each step while scanning the temperature so that the loop stabilizes
                   
):
    
    if data_title == '':
        raise ValueError('Always provide data_title')

    bench_mode = re.findall('[a-z-.]+' ,board_io( 'bench2 mode')[-1] )[1]
    if bench_mode != 'temp' and heater == 'bench2':
        raise ValueError('bench2 is not in the Thermal Control Mode')
        
    start_time = time.time()

    # Prepare the Temperature and Gain Current Ranges
    temp_range = np.arange(temp_max, temp_min + temp_step, temp_step) if temp_step < 0 else np.arange(temp_min, temp_max + temp_step, temp_step)    
    gc_range = np.arange(gc_max, gc_min + gc_step, gc_step) if gc_step < 0 else np.arange(gc_min, gc_max + gc_step, gc_step) 

    # Initialize the bench
    if len(k_bench) == 0: # if user has not defined it
        k_bench = get_pid(heater ='bench2')
        print('PID for bench: ' + str(k_bench))

    board_io( 'bench2 target %s' %temp_bench2) 
    board_io( 'bench2 pid %s %s %s' %(tuple(k_bench))) # set the PID
    board_io( 'bench2 enable '+ bench_enabled)     # Enable bench

    # Initialize the FSCS Heater
    if len(k_fscs) == 0: # if user has not defined it
        k_fscs = get_pid(heater ='fscs')
        print('PID for fscs: ' + str(k_fscs))

    board_io( 'fscs target %s' %temp_fscs) 
    board_io( 'fscs pid %s %s %s' %(tuple(k_fscs))) # set the PID
    board_io( 'fscs enable ' + fscs_enabled) # Enable the heater

    # Initialize the Comb Heater
    if len(k_comb) == 0: # if user has not defined it
        k_comb = get_pid(heater ='comb')
        print('PID for comb: ' + str(k_comb))

    board_io( 'comb target %s' %temp_comb) 
    board_io( 'comb pid %s %s %s' %(tuple(k_comb))) # set the PID
    board_io( 'comb enable ' + comb_enabled) # Enable the heater

    time_measured = []
    mpd_temp = []; power_temp = []; temp_diff = []; frequency_temp = []; gc = []; mod_i_oop_temp = []; mod_q_oop_temp= []
    gc_diff = []; frequency_read = 0 ; temp = []; 
    fscs_heat_temp = []; comb_heat_temp = []
    T_fscs_actual_temp = []; T_bench_actual_temp = []; T_comb_actual_temp = [] 


    power_meter.sense.power.dc.range.upper = 60/1000 # 60 mW, set to maximum range of optical power by PM100D

    for gc_write in gc_range:

        # change to the first value of temp_range and wait
        # so that this abrupt change will not cause any abnomalities
        board_io( '%s target %s' %(heater,temp_range[0]) )#

        gc_read = board_io( 'gc %s' %gc_write)#
        gc_read = gc_read[-1] # last element contains the GC value
        gc_read = float( re.findall('[0-9-.]+' , gc_read)[0] ) # extract the current from the returned string
        gc_diff.append( abs(gc_write-gc_read) ) # difference between the input and the read gc value
        gc.append(gc_read)

        time.sleep(10) # so that the spectrum stabilizes 

        power = []; mpd = []; frequency = []; mod_i_oop = []; mod_q_oop = []; 
        fscs_heat=[]; comb_heat=[]
        T_fscs_actual = []; T_bench_actual = []; T_comb_actual = [] 
        time_measured_temp = []

        for temp_write in temp_range:

            board_io( '%s target %s' %(heater,temp_write) )#

            time.sleep(bench_wait_time) # so that temperature stabilizes 
            
            time_measured_temp.append( time.strftime("%Y-%m-%d_%H:%M:%S") )

            # FSCS Actual Temperature
            T_fscs_actual_read = board_io( 'fscs actual')[-1]
            T_fscs_actual_read = float ( re.findall('[0-9-.]+' , T_fscs_actual_read)[0]) # extract the temperature from the returned string
            T_fscs_actual.append(T_fscs_actual_read)
            
            # bench2 Actual Temperature
            T_bench_actual_read = board_io( 'bench2 actual')[-1]
            T_bench_actual_read = float ( re.findall('[0-9-.]+' , T_bench_actual_read)[0]) # extract the temperature from the returned string
            T_bench_actual.append(T_bench_actual_read)
            
            # COMB Actual Temperature
            T_comb_actual_read = board_io( 'comb actual')[-1]
            T_comb_actual_read = float ( re.findall('[0-9-.]+' , T_comb_actual_read)[0]) # extract the temperature from the returned string
            T_comb_actual.append(T_comb_actual_read)
            
            T_actual_read ={'bench2' : T_bench_actual_read, 'fscs' : T_fscs_actual_read, 'comb' : T_comb_actual_read }[heater]

            temp_diff.append( abs(temp_write - T_actual_read) ) # difference between the input and the read temp value

            ### Measurements ###
            # Optical Power from Thorlabs PM100D
            power_read = power_meter.read * 1000 # converted to mW
            if power_read > 1000 : # if the PM100D is saturated
                power_read = 0

            power.append(power_read)

            # Optical Power and Peak frequency from Bristol 438A Wavelength Meter
            #power_bristol.append(float(bristol.getSimpleMsg(b':MEAS:SCAL:ALL?').split(', ')[3]))
            frequency_read = float(bristol.getSimpleMsg(b':MEAS:SCAL:ALL?').split(', ')[2])
            frequency.append(frequency_read)

            # Read oop, connected to mpd photodiode on TV0, adc to read may be 'oop' or 'ap'
            mpd_read = board_io( 'adc oop')[-1]
            mpd_read = float (re.findall('[0-9-.]+', mpd_read)[0]) # extract the mpd value from the returned string
            mpd.append(mpd_read) 

            # Read 'mod i oop' and 'mod q oop'
            mod_iq_read = board_io( 'mod iq oop')[-1]
            mod_i_oop_read = float(re.findall('[0-9-.]+', mod_iq_read)[0]); mod_q_oop_read = float(re.findall('[0-9-.]+', mod_iq_read)[-1])
            mod_i_oop.append(mod_i_oop_read); mod_q_oop.append(mod_q_oop_read); 
            
            fscs_heat_read = board_io( 'fscs heat')[-1]
            fscs_heat_read = float(re.findall('[0-9-.]+', fscs_heat_read)[0])
            fscs_heat.append(fscs_heat_read)

            comb_heat_read = board_io( 'comb heat')[-1]
            comb_heat_read = float(re.findall('[0-9-.]+', comb_heat_read)[0])
            comb_heat.append(comb_heat_read)

            print( '%.2f °C, Bench:%.2f °C, Comb:%.2f °C, FSCS:%.2f °C, %.1f mA P.current, %.1f mW Opt.Power, %.5f THz, %.1f mA GC' %(temp_write,T_bench_actual_read, T_comb_actual_read, T_fscs_actual_read, mpd_read, power_read, frequency_read, gc_read))


        mod_i_oop_temp.append(mod_i_oop); mod_q_oop_temp.append(mod_q_oop); 
        mpd_temp.append(mpd); power_temp.append(power); frequency_temp.append(frequency)
        fscs_heat_temp.append(fscs_heat); comb_heat_temp.append(comb_heat)
        T_fscs_actual_temp.append(T_fscs_actual); T_bench_actual_temp.append(T_bench_actual); T_comb_actual_temp.append(T_comb_actual)
        
        time_measured.append( time_measured_temp )


        elapsed_time = time.time() - start_time
        print('Elapsed time %.1f seconds (= %.2f hours)' %(elapsed_time, elapsed_time/3600) )

    print('Max difference between target & actual T = %.3f °C' %max(temp_diff))

    # Set the bench2 back to temp_min
    board_io( '%s target %s' %(heater, min(temp_range)))
    print('%s is set to %s' %(heater, min(temp_range) ) ) 

    # Turn off the GC
    gc_read = board_io( 'gc 0')[-1]
    gc_read = float( re.findall('[0-9-.]+' , gc_read)[0] ) # extract the current from the returned string
    print('Gain Current is %.3f mA now' %gc_read)
    elapsed_time = time.time() - start_time
    print('All took %.1f seconds (= %.2f hours)' %(elapsed_time, elapsed_time/3600) )

    # Transpose the measured arrays because the scan direction is Bias GC, Scan T
    [time_measured, mpd_temp, power_temp, frequency_temp, mod_i_oop_temp, mod_q_oop_temp] = transpose_lists([time_measured, mpd_temp, power_temp, frequency_temp, mod_i_oop_temp, mod_q_oop_temp])
    [T_fscs_actual_temp, T_bench_actual_temp, T_comb_actual_temp, fscs_heat_temp, comb_heat_temp] =  transpose_lists([T_fscs_actual_temp, T_bench_actual_temp, T_comb_actual_temp, fscs_heat_temp, comb_heat_temp])
    
    time_measured =  list(itertools.chain.from_iterable(time_measured))
    
    return [time_measured, temp_range, gc_range, mpd_temp, power_temp, frequency_temp, mod_i_oop_temp, mod_q_oop_temp, T_fscs_actual_temp, T_bench_actual_temp, T_comb_actual_temp, fscs_heat_temp, comb_heat_temp]

# FSCS Temperature Scans

In [16]:
def scan_fscs_temp(data_title='',
                   temp_min = 25, temp_max = 26, temp_step = -1, # fscs temperature range
                   k_fscs = [], # fscs PID
                   temp_bench2 = 25, k_bench= [], # bench2 temperature and PID
                   temp_comb = 25, k_comb = [], # comb temperature and PID
                   gc_min = 25 , gc_max = 50, gc_step = -25, # Set min, max gain current, the step size in mA
                   wait_time = 3, # wait time between each step while scanning the temperature so that the loop stabilizes
):
    
    if data_title == '':
        raise ValueError('Always provide data_title')
        
    start_time = time.time()

    # Prepare the Temperature and Gain Current Ranges
    temp_range = np.arange(temp_max, temp_min + temp_step, temp_step) if temp_step < 0 else np.arange(temp_min, temp_max + temp_step, temp_step)    
    gc_range = np.arange(gc_max, gc_min + gc_step, gc_step) if gc_step < 0 else np.arange(gc_min, gc_max + gc_step, gc_step) 

    if len(temp_range) < 2 or len(gc_range) < 2:
        raise ValueError('Length of Temperature and GC Ranges need to be at least 2')
        
    temp_fscs = temp_range[0] # target is the first temperature value

    # Initialize the FSCS Heater
    if len(k_fscs) == 0: # if user has not defined it
        k_fscs = get_pid(heater ='fscs')
        print('PID for fscs: ' + str(k_fscs))

    board_io( 'fscs target %s' %temp_fscs) 
    board_io( 'fscs pid %s %s %s' %(tuple(k_fscs))) # set the PID
    board_io( 'fscs enable ' + fscs_enabled) # Enable the heater
    
    # Initialize the bench
    if len(k_bench) == 0: # if user has not defined it
        k_bench = get_pid(heater ='bench2')
        print('PID for bench: ' + str(k_bench))

    board_io( 'bench2 target %s' %temp_bench2) 
    board_io( 'bench2 pid %s %s %s' %(tuple(k_bench))) # set the PID
    board_io( 'bench2 enable '+ bench_enabled)     # Enable bench

    # Initialize the Comb Heater
    if len(k_comb) == 0: # if user has not defined it
        k_comb = get_pid(heater ='comb')
        print('PID for comb: ' + str(k_comb))

    board_io( 'comb target %s' %temp_comb) 
    board_io( 'comb pid %s %s %s' %(tuple(k_comb))) # set the PID
    board_io( 'comb enable ' + comb_enabled) # Enable the heater

    mpd_temp = []; power_temp = []; temp_diff = []; frequency_temp = []; gc = []; mod_i_oop_temp = []; mod_q_oop_temp= []
    gc_diff = []; frequency_read = 0 ; temp = []; 
    fscs_heat_temp = []; comb_heat_temp = []
    T_fscs_actual_temp = []; T_bench_actual_temp = []; T_comb_actual_temp = [] 

    power_meter.sense.power.dc.range.upper = 60/1000 # 60 mW, set to maximum range of optical power by PM100D

    for gc_write in gc_range:

        # change to the first value of temp_range and wait
        # so that this abrupt change will not cause any abnomalities
        board_io( 'fscs target %s' %temp_range[0])#

        gc_read = board_io( 'gc %s' %gc_write)#
        gc_read = gc_read[-1] # last element contains the GC value
        gc_read = float( re.findall('[0-9-.]+' , gc_read)[0] ) # extract the current from the returned string
        gc_diff.append( abs(gc_write-gc_read) ) # difference between the input and the read gc value
        gc.append(gc_read)

        time.sleep(10) # so that the spectrum stabilizes 

        power = []; mpd = []; frequency = []; mod_i_oop = []; mod_q_oop = []; 
        fscs_heat=[]; comb_heat=[]
        T_fscs_actual = []; T_bench_actual = []; T_comb_actual = [] 


        for temp_write in temp_range:

            board_io( 'fscs target %s' %temp_write)#

            time.sleep(wait_time) # so that temperature stabilizes 

            # FSCS Actual Temperature
            T_fscs_actual_read = board_io( 'fscs actual')[-1]
            T_fscs_actual_read = float ( re.findall('[0-9-.]+' , T_fscs_actual_read)[0]) # extract the temperature from the returned string
            T_fscs_actual.append(T_fscs_actual_read)

            temp_diff.append( abs(temp_write - T_fscs_actual_read) ) # difference between the input and the read temp value
            
            # bench2 Actual Temperature
            T_bench_actual_read = board_io( 'bench2 actual')[-1]
            T_bench_actual_read = float ( re.findall('[0-9-.]+' , T_bench_actual_read)[0]) # extract the temperature from the returned string
            T_bench_actual.append(T_bench_actual_read)

            # COMB Actual Temperature
            T_comb_actual_read = board_io( 'comb actual')[-1]
            T_comb_actual_read = float ( re.findall('[0-9-.]+' , T_comb_actual_read)[0]) # extract the temperature from the returned string
            T_comb_actual.append(T_comb_actual_read)

            ### Measurements ###
            # Optical Power from Thorlabs PM100D
            power_read = power_meter.read * 1000 # converted to mW
            if power_read > 1000 : # if the PM100D is saturated
                power_read = 0

            power.append(power_read)

            # Optical Power and Peak frequency from Bristol 438A Wavelength Meter
            #power_bristol.append(float(bristol.getSimpleMsg(b':MEAS:SCAL:ALL?').split(', ')[3]))
            frequency_read = float(bristol.getSimpleMsg(b':MEAS:SCAL:ALL?').split(', ')[2])
            frequency.append(frequency_read)

            # Read oop, connected to mpd photodiode on TV0, adc to read may be 'oop' or 'ap'
            mpd_read = board_io( 'adc oop')[-1]
            mpd_read = float (re.findall('[0-9-.]+', mpd_read)[0]) # extract the mpd value from the returned string
            mpd.append(mpd_read) 

            # Read 'mod i oop' and 'mod q oop'
            mod_iq_read = board_io( 'mod iq oop')[-1]
            mod_i_oop_read = float(re.findall('[0-9-.]+', mod_iq_read)[0]); mod_q_oop_read = float(re.findall('[0-9-.]+', mod_iq_read)[-1])
            mod_i_oop.append(mod_i_oop_read); mod_q_oop.append(mod_q_oop_read); 
            
            fscs_heat_read = board_io( 'fscs heat')[-1]
            fscs_heat_read = float(re.findall('[0-9-.]+', fscs_heat_read)[0])
            fscs_heat.append(fscs_heat_read)

            comb_heat_read = board_io( 'comb heat')[-1]
            comb_heat_read = float(re.findall('[0-9-.]+', comb_heat_read)[0])
            comb_heat.append(comb_heat_read)

            print( '%.2f mA P.current, %.2f mW Opt. Power, %.5f THz, %.1f mA GC, target %.2f °C (actual %.2f °C)' %(mpd_read, power_read, frequency_read, gc_read, temp_write,T_fscs_actual_read))

        mod_i_oop_temp.append(mod_i_oop); mod_q_oop_temp.append(mod_q_oop); 
        mpd_temp.append(mpd); power_temp.append(power); frequency_temp.append(frequency)
        
        T_fscs_actual_temp.append(T_fscs_actual); T_bench_actual_temp.append(T_bench_actual); T_comb_actual_temp.append(T_comb_actual)
        fscs_heat_temp.append(fscs_heat); comb_heat_temp.append(comb_heat)

        elapsed_time = time.time() - start_time
        print('Elapsed time %.1f seconds (= %.2f hours)' %(elapsed_time, elapsed_time/3600) )

    print('Max difference between target & actual T = %.3f °C' %max(temp_diff))

    # Set the FSCS back to temp_min
    board_io( 'fscs target %s' %min(temp_range))
    print('FSCS is set to %s' %min(temp_range))

    # Turn off the GC
    gc_read = board_io( 'gc 0')[-1]
    gc_read = float( re.findall('[0-9-.]+' , gc_read)[0] ) # extract the current from the returned string
    print('Gain Current is %.3f mA now' %gc_read)
    elapsed_time = time.time() - start_time
    print('All took %.1f seconds (= %.2f hours)' %(elapsed_time, elapsed_time/3600) )

    # Transpose the measured arrays because the scan direction is Bias GC, Scan T
    [mpd_temp, power_temp, frequency_temp, mod_i_oop_temp, mod_q_oop_temp] = transpose_lists([mpd_temp, power_temp, frequency_temp, mod_i_oop_temp, mod_q_oop_temp])
    [T_fscs_actual_temp, T_bench_actual_temp, T_comb_actual_temp, fscs_heat_temp, comb_heat_temp] =  transpose_lists([T_fscs_actual_temp, T_bench_actual_temp, T_comb_actual_temp, fscs_heat_temp, comb_heat_temp])
    
    return [temp_range, gc_range, mpd_temp, power_temp, frequency_temp, mod_i_oop_temp, mod_q_oop_temp, T_fscs_actual_temp, T_bench_actual_temp, T_comb_actual_temp, fscs_heat_temp, comb_heat_temp]

# Export PIT Data

In [3]:
def export_pit_data(time_measured, temp_range, gc_range, mpd_temp, power_temp, frequency_temp, mod_i_oop_temp, mod_q_oop_temp, T_fscs_actual_temp, T_bench_actual_temp, T_comb_actual_temp, fscs_heat_temp, comb_heat_temp, 
                    data_title = '',

                   ):
    output_labels  = ['Time', 'Temperature (°C)', 'Gain Current (mA)', 'MPD (mA)', 'Optical Power (mW)', 'Frequency (THz)', 
                      'Mod i oop', 'Mod q oop', 'FSCS Temperature (°C)', 'bench2 Temperature (°C)', 'Comb Temperature (°C)',
                      'FSCS Heat (mW)', 'Comb Heat (mW)'
                     ]
    
    if data_title == '' or output_labels == '':
        raise ValueError('Always provide data_title and output_labels ')
        
    temp_range_expanded = np.repeat(temp_range, len(gc_range))
    gc_range_expanded = list(gc_range) * len(temp_range) #np.repeat(gc_range,len(temp_range))
    mpd_temp_expanded = list(itertools.chain.from_iterable(mpd_temp)) # flatten the list to convert to a dataframe column
    power_temp_expanded = list(itertools.chain.from_iterable(power_temp)) # flatten the list to convert to a dataframe column
    frequency_temp_expanded = list(itertools.chain.from_iterable(frequency_temp)) # flatten the list to convert to a dataframe column
    
    T_fscs_actual_temp_expanded = list(itertools.chain.from_iterable(T_fscs_actual_temp))
    T_bench_actual_temp_expanded = list(itertools.chain.from_iterable(T_bench_actual_temp))
    T_comb_actual_temp_expanded = list(itertools.chain.from_iterable(T_comb_actual_temp))

    mod_i_oop_temp_expanded = list(itertools.chain.from_iterable(mod_i_oop_temp))
    mod_q_oop_temp_expanded = list(itertools.chain.from_iterable(mod_q_oop_temp))
    
    fscs_heat_temp_expanded = list(itertools.chain.from_iterable(fscs_heat_temp))
    comb_heat_temp_expanded = list(itertools.chain.from_iterable(comb_heat_temp))




    output = [time_measured, temp_range_expanded, gc_range_expanded, mpd_temp_expanded, power_temp_expanded, frequency_temp_expanded,
             mod_i_oop_temp_expanded, mod_q_oop_temp_expanded,
              T_fscs_actual_temp_expanded, T_bench_actual_temp_expanded, T_comb_actual_temp_expanded,
             fscs_heat_temp_expanded, comb_heat_temp_expanded
             ]
    df_dict = dict(zip( output_labels, output))
    
    data_pd = pd.DataFrame( df_dict, columns = output_labels) 

    export_pd(data_pd, folder_address = 'Outputs', file_name = data_title, header=True)

# Print Heater Properties

In [13]:
def print_thermal_state():
    
    mod_iq_oop = board_io( 'mod iq oop')[-1]
    mod_i_oop = re.findall('[0-9-.]+', mod_iq_oop)[0];
    
    bench_control = board_io( 'bench2 control') [-1]
    bench_control = float(re.findall('[0-9-.]+', bench_control)[0])
    
    bench_mode = re.findall('[a-z-.]+' ,board_io( 'bench2 mode')[-1] )[1]
    

    # Heater Temperatures
    print('bench2 = '+re.findall('[0-9-.]+' , board_io( 'bench2 actual')[-1])[0]  + " °C" + '; Control = %.3f A (Mode: %s)' %(bench_control,bench_mode))
    print('i = ' + mod_i_oop)
    
    comb_heat = float(re.findall('[0-9-.]+' , board_io( 'comb heat')[-1])[0])
    comb_temp = float(re.findall('[0-9-.]+' , board_io( 'comb actual')[-1])[0])
    fscs_heat = float(re.findall('[0-9-.]+' , board_io( 'fscs heat')[-1])[0])
    fscs_temp = float(re.findall('[0-9-.]+' , board_io( 'fscs actual')[-1])[0])
    print('Comb = %.3f °C (%.3f mW)' %(comb_temp, comb_heat)) 
    print('FSCS = %.3f °C (%.3f mW)' %(fscs_temp, fscs_heat)) 
    
def print_heats():
    # Heater Powers
    print('Comb = ' + re.findall('[0-9-.]+' , board_io( 'comb heat')[-1])[0] +' mW') 
    print('FSCS = ' + re.findall('[0-9-.]+' , board_io( 'fscs heat')[-1])[0] +' mW')

# Plotting Temperature and Gain Chip Current Sweep Results
Photocurrent in a photodiode, Optical Power Meter and Peak Frequency.  
Also exports to slides in a powerpoint file.

In [4]:
def plot_pit_sweeps(temp, gc, mpd_temp, power_temp, frequency_temp, mod_i_oop_temp, mod_q_oop_temp,
                    temp_step_plot_factor=1, gc_step_plot_factor=1, # to skip some of the temperature and gc values to be plotted
                    file_name_powerpoint = '', # export the figures to slides if file_name_powerpoint is specified
                    data_title = '', 
                    folder_address = 'Outputs'
                   ):
    

    gc_axis_label = 'Gain Current (mA)'; temp_axis_label = 'Temperature (°C)' ;  mod_ip_oop_axis_label = 'Mod iq oop'
    mpd_axis_label = 'Photocurrent (mA)'; power_axis_label = 'Optical Power (mW)'; frequency_axis_label = 'Frequency (THz)'
    delta_mpd_axis_label = 'ΔPhotocurrent (mA)'; delta_power_axis_label = 'ΔOptical Power (mW)'; delta_frequency_axis_label = 'ΔFrequency (THz)'    
    
    figsize = (20,8); fontsize = 20
    fig, ax = plt.subplots(figsize=figsize)
    plt.rcParams['axes.linewidth'] = 3 #set the value globally
    
    line_kwargs = {'ls' :'-', 'lw' : 1, 'ms' : 1, 'marker' : '', 'markeredgewidth' : 2, 'markerfacecolor' : 'None'}
    grid_kwargs = {'color':'gray', 'ls': '--', 'lw': 1, 'axis' : 'both'}
    tick_params = {'direction' : 'in', 'labelsize' : fontsize/1.5}
    temp_color_map = []
    
    if temp_step_plot_factor > len(temp):
        ValueError('temp_step_plot_factor can not be larger than the number of temperature points')
    if gc_step_plot_factor > len(gc):
        ValueError('gc_step_plot_factor can not be larger than the number of GC points')


    for i in range(0, len(temp), temp_step_plot_factor):

        temp_plot_range =  temp[0:len(temp):temp_step_plot_factor] # the temperature values being plotted
        
        try :

            if temp[0] < temp[1]: # if temperature decreases in the scan
                RGB = ( i/len(temp), 0, 1 - i/len(temp) ) # Lowest T = blue
            else :                                                         
                RGB = ( 1 - i/len(temp), 0, i/len(temp) ) # Lowest T = blue
            # Setting up a colormap that's same as the line colors
            temp_color_map_i = (temp[i] - min(temp_plot_range))/(max(temp_plot_range) - min(temp_plot_range))
            temp_color_map.append(  (temp_color_map_i, RGB ) ) 
            
            line_kwargs['color'] = RGB
        except :
            pass
        
        
        line_kwargs['label'] = '%.1f' %temp[i]
        
        ax=plt.subplot(131)
        ax.plot(gc, mpd_temp[i], **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
        #ax.legend(title = temp_axis_label, loc='best', fontsize = fontsize/1.5)
        ax.set_xlabel(gc_axis_label, fontsize = fontsize); ax.set_ylabel(mpd_axis_label, fontsize = fontsize)
    
        ax=plt.subplot(132)
        ax.plot(gc, power_temp[i],  **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
        #ax.legend(title = temp_axis_label, loc='best', fontsize = fontsize/1.5)
        ax.set_xlabel(gc_axis_label, fontsize = fontsize); ax.set_ylabel(power_axis_label, fontsize = fontsize)
        
        ax=plt.subplot(133)
        ax.plot(gc, frequency_temp[i],  **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
        #ax.legend(title = temp_axis_label, loc='best', fontsize = fontsize/1.5)
        ax.set_xlabel(gc_axis_label, fontsize = fontsize); ax.set_ylabel(frequency_axis_label, fontsize = fontsize)
    
    try :
        if temp[1] < temp[0]: # if temperature is decreasing during the scan
            temp_color_map = temp_color_map[::-1] # invert the order of temperature color map

        my_cmap = mpl.colors.LinearSegmentedColormap.from_list( 'mycolors', temp_color_map )
        sm = plt.cm.ScalarMappable(cmap = my_cmap, norm = plt.Normalize( vmin = temp[0], vmax = temp[i] ) )
        # fake up the array of the scalar mappable.
        sm._A = []
        
        plt.colorbar(sm, label=temp_axis_label)
    except:
        pass
    
    plt.tight_layout(pad=0.2)
    plt.savefig(folder_address + '\\' + temp_axis_label + '_sweep.png', bbox_inches='tight')
    plt.show()
    
    fig, ax = plt.subplots(figsize=figsize)
    gc_color_map = []
    for i in range(0, len(gc), gc_step_plot_factor):
        
        RGB = ( 0, 1 - i/len(gc), 0 )
        line_kwargs['color'] = RGB
        line_kwargs['label'] = '%.1f' %gc[i]
       
        ax=plt.subplot(141)
        ax.plot(temp, [mpd_gc[i] for mpd_gc in mpd_temp],  **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
        #ax.legend(title = gc_axis_label,loc='best', fontsize = fontsize/1.5)
        ax.set_xlabel(temp_axis_label, fontsize = fontsize); ax.set_ylabel(mpd_axis_label, fontsize = fontsize)
        
        ax=plt.subplot(142)
        ax.plot(temp, [power_gc[i] for power_gc in power_temp], **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
        #ax.legend(title = gc_axis_label,loc='best', fontsize = fontsize/1.5)
        ax.set_xlabel(temp_axis_label, fontsize = fontsize); ax.set_ylabel(power_axis_label, fontsize = fontsize)
        
        ax=plt.subplot(143)
        ax.plot(temp, [mod_i_oop_gc[i] for mod_i_oop_gc in mod_i_oop_temp] ,  **line_kwargs); ax.grid(**grid_kwargs); #ax.tick_params(**tick_params)
        
        RGB_mod_q = ( 0, 0.5, 1 - i/len(gc) )
        line_kwargs['color'] = RGB_mod_q # use different colors for mod_i and mod_q
        ax.plot(temp, [mod_q_oop_gc[i] for mod_q_oop_gc in mod_q_oop_temp] ,  **line_kwargs); ax.grid(**grid_kwargs); #ax.tick_params(**tick_params)
        #ax.set_xlim([46,54]) # set the x axis Temperature range on demand

        #ax.legend(title = gc_axis_label,loc='best', fontsize = fontsize/1.5)
        ax.set_xlabel(temp_axis_label, fontsize = fontsize); ax.set_ylabel(mod_ip_oop_axis_label, fontsize = fontsize)
        
        ax=plt.subplot(144)
        line_kwargs['color'] = RGB
        ax.plot(temp, [frequency_gc[i] for frequency_gc in frequency_temp] ,  **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
        #ax.legend(title = gc_axis_label,loc='best', fontsize = fontsize/1.5)
        ax.set_xlabel(temp_axis_label, fontsize = fontsize); ax.set_ylabel(frequency_axis_label, fontsize = fontsize)
        
        # Setting up a colormap that's same as the line colors
        gc_color_map.append( ( (gc[i] - min(gc))/(max(gc) - min(gc)), RGB )) 
      
    try :
        if gc[1] < gc[0]: # if gain chip current decreases during the scan
            gc_color_map = gc_color_map[::-1] # invert the order of gc color map
    except:
        pass
    
    my_cmap = mpl.colors.LinearSegmentedColormap.from_list( 'mycolors', gc_color_map )

    sm = plt.cm.ScalarMappable(cmap = my_cmap, norm = plt.Normalize( vmin = gc[0], vmax = gc[i] ) )
    
    # fake up the array of the scalar mappable.
    sm._A = []
    
    plt.colorbar(sm, label=gc_axis_label)
    plt.tight_layout(pad=0.2)
    plt.savefig(folder_address + '\\' + gc_axis_label + '_sweep.png', bbox_inches='tight')
    plt.show()
    
    
    # Plot the differentials of the data as a function of temperature
    fig, ax = plt.subplots(figsize=figsize)
    gc_color_map = []
    for i in range(0, len(gc), gc_step_plot_factor):
        
        RGB = ( 0, 1 - i/len(gc), 0 )
        line_kwargs['color'] = RGB
        line_kwargs['label'] = '%.1f' %gc[i]
       
        ax=plt.subplot(131)
        ax.plot(temp[1:], np.diff([mpd_gc[i] for mpd_gc in mpd_temp]), **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
        #ax.legend(title = gc_axis_label,loc='best', fontsize = fontsize/1.5)
        ax.set_xlabel(temp_axis_label, fontsize = fontsize); ax.set_ylabel(delta_mpd_axis_label, fontsize = fontsize)
        
        ax=plt.subplot(132)
        ax.plot(temp[1:], np.diff([power_gc[i] for power_gc in power_temp]), **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
        #ax.legend(title = gc_axis_label,loc='best', fontsize = fontsize/1.5)
        ax.set_xlabel(temp_axis_label, fontsize = fontsize); ax.set_ylabel(delta_power_axis_label, fontsize = fontsize)
        ax.set_ylim([-0.5,0.5])
        
        ax=plt.subplot(133)
        ax.plot(temp[1:], np.diff([frequency_gc[i] for frequency_gc in frequency_temp]), **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
        #ax.legend(title = gc_axis_label,loc='best', fontsize = fontsize/1.5)
        ax.set_xlabel(temp_axis_label, fontsize = fontsize); ax.set_ylabel(delta_frequency_axis_label, fontsize = fontsize)
        
        # Setting up a colormap that's same as the line colors
        gc_color_map.append( ( (gc[i] - min(gc))/(max(gc) - min(gc)), RGB )) 
      
    try :
        if gc[1] < gc[0]: # if gain chip current decreases during the scan
            gc_color_map = gc_color_map[::-1] # invert the order of gc color map
    except:
        pass
    
    
    my_cmap = mpl.colors.LinearSegmentedColormap.from_list( 'mycolors', gc_color_map )

    sm = plt.cm.ScalarMappable(cmap = my_cmap, norm = plt.Normalize( vmin = gc[0], vmax = gc[i] ) )
    # fake up the array of the scalar mappable.
    sm._A = []
    
    plt.colorbar(sm, label=gc_axis_label)#, cax=cbaxes,  orientation = 'horizontal', ticklocation = 'top')
    plt.tight_layout(pad=0.2)
    plt.savefig(folder_address + '\\' + gc_axis_label + '_sweep_differential.png', bbox_inches='tight')
    plt.show()
    
    # 2D colormaps as a function of Temperature and GC
    mpd_temp_vflipped = [sublist[::-1] for sublist in mpd_temp ] # flip around the vertical axis to plot gc in the increasing direction
    power_temp_vflipped = [sublist[::-1] for sublist in power_temp ] # flip around the vertical axis to plot gc in the increasing direction
    frequency_temp_vflipped = [sublist[::-1] for sublist in frequency_temp ] # flip around the vertical axis to plot gc in the increasing direction
     
    if temp[1] < temp[0]: # if temperature decreases during the scan
        # flip around the horizontal axis to plot temperature in the increasing direction
        mpd_temp_vhflipped = mpd_temp_vflipped[::-1] 
        power_temp_vhflipped = power_temp_vflipped[::-1]
        frequency_temp_vhflipped = frequency_temp_vflipped[::-1]
    else:
        mpd_temp_vhflipped = mpd_temp_vflipped
        power_temp_vhflipped = power_temp_vflipped
        frequency_temp_vhflipped = frequency_temp_vflipped

    colormap_kwargs = {}
    colormap_figures = []
    
    plt.figure(figsize = (10,8))
    plt.imshow(mpd_temp_vhflipped, extent=[min(gc), max(gc), min(temp), max(temp)], cmap='gray',
              origin='lowest', aspect='auto')
            #vmin=0, vmax=0.35, origin='lowest', aspect='auto')
    plt.xlabel(gc_axis_label, fontsize=fontsize); plt.ylabel(temp_axis_label, fontsize=fontsize)
    plt.xticks(fontsize=fontsize); plt.yticks(fontsize=fontsize)
    plt.suptitle(mpd_axis_label, fontsize=fontsize)
    cbar = plt.colorbar()
    cbar.set_label(mpd_axis_label, size=fontsize/1.5); cbar.ax.tick_params(labelsize=fontsize/1.5)
    plt.savefig(folder_address + '\\' + mpd_axis_label + '.png', bbox_inches='tight')
    colormap_figures.append(mpd_axis_label + '.png')
    plt.show()
    
    
    plt.figure(figsize = (10,8))
    plt.imshow(power_temp_vhflipped, extent=[min(gc), max(gc), min(temp), max(temp)], cmap='gray',
                origin='lowest', aspect='auto')
            #vmin=0, vmax=15, origin='lowest', aspect='auto')
    plt.xlabel(gc_axis_label, fontsize=fontsize); plt.ylabel(temp_axis_label , fontsize=fontsize)
    plt.xticks(fontsize=fontsize); plt.yticks(fontsize=fontsize)
    plt.suptitle(power_axis_label,fontsize=fontsize)
    cbar = plt.colorbar()
    cbar.set_label(power_axis_label, size=fontsize/1.5); cbar.ax.tick_params(labelsize=fontsize/1.5)
    
    plt.savefig(folder_address + '\\' + power_axis_label + '.png', bbox_inches='tight')
    colormap_figures.append(power_axis_label + '.png')
    plt.show()
    
    plt.figure(figsize = (10,8))
    plt.imshow(frequency_temp_vhflipped, extent=[min(gc), max(gc), min(temp), max(temp)], cmap='RdBu',
                origin='lowest', aspect='auto')
               #vmin=194.18, vmax=194.25, origin='lowest', aspect='auto')
    plt.xlabel(gc_axis_label, fontsize=fontsize); plt.ylabel(temp_axis_label , fontsize=fontsize)
    plt.xticks(fontsize=fontsize); plt.yticks(fontsize=fontsize)
    plt.suptitle(frequency_axis_label,fontsize=fontsize)
    cbar = plt.colorbar()
    cbar.set_label(frequency_axis_label, size=fontsize/1.5); cbar.ax.tick_params(labelsize=fontsize/1.5)
    
    plt.savefig(folder_address + '\\' + frequency_axis_label + '.png', bbox_inches='tight')
    colormap_figures.append(frequency_axis_label + '.png')
    plt.show()
    
    if file_name_powerpoint != '' :
        images_to_powerpoint(file_name_powerpoint, [temp_axis_label + '_sweep.png'], slide_title = 'Temperature sweep', slide_label = data_title)
        images_to_powerpoint(file_name_powerpoint, [gc_axis_label + '_sweep.png'], slide_title = 'GC sweep', slide_label = data_title)
        images_to_powerpoint(file_name_powerpoint, [gc_axis_label + '_sweep_differential.png'], slide_title = 'GC sweep (Differential)', slide_label = data_title)

        images_to_powerpoint(file_name_powerpoint, colormap_figures, slide_title = '2D Plotting', slide_label = data_title)
    
    return ax

In [6]:
# to plot 2 identical datasets with opposite temperature scan directions
def plot_multiple_pit_sweeps(temp, gc, mpd_temp, power_temp, frequency_temp, 
                    temp_step_plot_factor=1, gc_step_plot_factor=1, # to skip some of the temperature and gc values to be plotted
                    file_name_powerpoint = '', # export the figures to slides if file_name_powerpoint is specified
                    data_title = '', 
                    folder_address = 'Outputs'
                   ):

    gc_axis_label = 'Gain Current (mA)'; mpd_axis_label = 'Photocurrent (mA)'; temp_axis_label = 'Temperature (°C)'
    power_axis_label = 'Optical Power (mW)'; frequency_axis_label = 'Frequency (THz)'
    figsize = (20,8); fontsize = 20
    fig, ax = plt.subplots(figsize=figsize)
    plt.rcParams['axes.linewidth'] = 3 #set the value globally
    
    line_kwargs = {'ls' :'-', 'lw' : 1, 'ms' : 1, 'marker' : '', 'markeredgewidth' : 2, 'markerfacecolor' : 'None'}
    grid_kwargs = {'color':'gray', 'ls': '--', 'lw': 1, 'axis' : 'both'}
    tick_params = {'direction' : 'in', 'labelsize' : fontsize/1.5}
    temp_color_map = []
    
    temp2 = temp[1]
    temp = temp[0]
    
    mpd_temp2 = mpd_temp[1]
    mpd_temp = mpd_temp[0]
    
    power_temp2 = power_temp[1]
    power_temp = power_temp[0]

    frequency_temp2 = frequency_temp[1]
    frequency_temp = frequency_temp[0]
    

    for i in range(0, len(temp), temp_step_plot_factor):

        temp_plot_range =  temp[0:len(temp):temp_step_plot_factor] # the temperature values being plotted
        
        if temp[0] < temp[1]: # if temperature decreases in the scan
            RGB = ( i/len(temp), 0, 1 - i/len(temp) ) # Lowest T = blue
        else :                                                         
            RGB = ( 1 - i/len(temp), 0, i/len(temp) ) # Lowest T = blue
        
        # Setting up a colormap that's same as the line colors
        temp_color_map_i = (temp[i] - min(temp_plot_range))/(max(temp_plot_range) - min(temp_plot_range))
        temp_color_map.append(  (temp_color_map_i, RGB ) ) 
        
        line_kwargs['color'] = RGB
        line_kwargs['label'] = '%.1f' %temp[i]
        
        ax=plt.subplot(131)
        ax.plot(gc, mpd_temp[i], **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
        #ax.legend(title = temp_axis_label, loc='best', fontsize = fontsize/1.5)
        ax.set_xlabel(gc_axis_label, fontsize = fontsize); ax.set_ylabel(mpd_axis_label, fontsize = fontsize)
    
        ax=plt.subplot(132)
        ax.plot(gc, power_temp[i],  **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
        #ax.legend(title = temp_axis_label, loc='best', fontsize = fontsize/1.5)
        ax.set_xlabel(gc_axis_label, fontsize = fontsize); ax.set_ylabel(power_axis_label, fontsize = fontsize)
        
        ax=plt.subplot(133)
        ax.plot(gc, frequency_temp[i],  **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
        #ax.legend(title = temp_axis_label, loc='best', fontsize = fontsize/1.5)
        ax.set_xlabel(gc_axis_label, fontsize = fontsize); ax.set_ylabel(frequency_axis_label, fontsize = fontsize)
    
    if temp[1] < temp[0]: # if temperature is decreasing during the scan
        temp_color_map = temp_color_map[::-1] # invert the order of temperature color map
        
    my_cmap = mpl.colors.LinearSegmentedColormap.from_list( 'mycolors', temp_color_map )
    sm = plt.cm.ScalarMappable(cmap = my_cmap, norm = plt.Normalize( vmin = temp[0], vmax = temp[i] ) )
    # fake up the array of the scalar mappable.
    sm._A = []
    
    plt.colorbar(sm, label=temp_axis_label)
    plt.savefig(folder_address + '\\' + temp_axis_label + '_sweep.png', bbox_inches='tight')
    plt.subplots_adjust(wspace=0.30)
    plt.show()
    
    fig, ax = plt.subplots(figsize=figsize)
    gc_color_map = []
    for i in range(0, len(gc), gc_step_plot_factor):
        
        RGB = ( 0, 1 - i/len(gc), 0 )
        RGB2 = ( 0, 0, 1 - i/len(gc) )
        line_kwargs['color'] = RGB
        line_kwargs['label'] = '%.1f' %gc[i]
        
        line_kwargs2 = line_kwargs.copy()
        line_kwargs2['color'] = RGB2

        ax=plt.subplot(131)
        ax.plot(temp, [mpd_gc[i] for mpd_gc in mpd_temp], **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
        ax.plot(temp2, [mpd_gc2[i] for mpd_gc2 in mpd_temp2], **line_kwargs2);
        #ax.legend(title = gc_axis_label,loc='best', fontsize = fontsize/1.5)
        ax.set_xlabel(temp_axis_label, fontsize = fontsize); ax.set_ylabel(mpd_axis_label, fontsize = fontsize)
        
        ax=plt.subplot(132)
        ax.plot(temp, [power_gc[i] for power_gc in power_temp], **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
        ax.plot(temp2, [power_gc2[i] for power_gc2 in power_temp2], **line_kwargs2)
        #ax.legend(title = gc_axis_label,loc='best', fontsize = fontsize/1.5)
        ax.set_xlabel(temp_axis_label, fontsize = fontsize); ax.set_ylabel(power_axis_label, fontsize = fontsize)
        #ax.set_xlim([22,24])
        
        ax=plt.subplot(133)
        ax.plot(temp, [frequency_gc[i] for frequency_gc in frequency_temp], **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
        ax.plot(temp2, [frequency_gc2[i] for frequency_gc2 in frequency_temp2], **line_kwargs2);
        #ax.legend(title = gc_axis_label,loc='best', fontsize = fontsize/1.5)
        ax.set_xlabel(temp_axis_label, fontsize = fontsize); ax.set_ylabel(frequency_axis_label, fontsize = fontsize)
        
        # Setting up a colormap that's same as the line colors
        gc_color_map.append( ( (gc[i] - min(gc))/(max(gc) - min(gc)), RGB )) 
      
    try :
        if gc[1] < gc[0]: # if gain chip current decreases during the scan
            gc_color_map = gc_color_map[::-1] # invert the order of gc color map
    except:
        pass
    
    my_cmap = mpl.colors.LinearSegmentedColormap.from_list( 'mycolors', gc_color_map )

    sm = plt.cm.ScalarMappable(cmap = my_cmap, norm = plt.Normalize( vmin = gc[0], vmax = gc[i] ) )
    # fake up the array of the scalar mappable.
    sm._A = []
    
    plt.colorbar(sm, label=gc_axis_label)#, cax=cbaxes,  orientation = 'horizontal', ticklocation = 'top')
    plt.savefig(folder_address + '\\' + gc_axis_label + '_sweep.png', bbox_inches='tight')
    plt.subplots_adjust(wspace=0.30)
    plt.show()
    
    # 2D colormaps as a function of Temperature and GC
    mpd_temp_vflipped = [sublist[::-1] for sublist in mpd_temp ] # flip around the vertical axis to plot gc in the increasing direction
    power_temp_vflipped = [sublist[::-1] for sublist in power_temp ] # flip around the vertical axis to plot gc in the increasing direction
    frequency_temp_vflipped = [sublist[::-1] for sublist in frequency_temp ] # flip around the vertical axis to plot gc in the increasing direction
     
    if temp[1] < temp[0]: # if temperature decreases during the scan
        # flip around the horizontal axis to plot temperature in the increasing direction
        mpd_temp_vhflipped = mpd_temp_vflipped[::-1] 
        power_temp_vhflipped = power_temp_vflipped[::-1]
        frequency_temp_vhflipped = frequency_temp_vflipped[::-1]
    else:
        mpd_temp_vhflipped = mpd_temp_vflipped
        power_temp_vhflipped = power_temp_vflipped
        frequency_temp_vhflipped = frequency_temp_vflipped

    colormap_kwargs = {}
    colormap_figures = []
    
    plt.figure(figsize = (10,8))
    plt.imshow(mpd_temp_vhflipped, extent=[min(gc), max(gc), min(temp), max(temp)], cmap='gray',
              origin='lowest', aspect='auto')
            #vmin=0, vmax=0.35, origin='lowest', aspect='auto')
    plt.xlabel(gc_axis_label, fontsize=fontsize); plt.ylabel(temp_axis_label, fontsize=fontsize)
    plt.xticks(fontsize=fontsize); plt.yticks(fontsize=fontsize)
    plt.suptitle(mpd_axis_label, fontsize=fontsize)
    plt.colorbar(label= mpd_axis_label)
    plt.savefig(folder_address + '\\' + mpd_axis_label + '.png', bbox_inches='tight')
    colormap_figures.append(mpd_axis_label + '.png')
    plt.show()
    
    
    plt.figure(figsize = (10,8))
    plt.imshow(power_temp_vhflipped, extent=[min(gc), max(gc), min(temp), max(temp)], cmap='gray',
                origin='lowest', aspect='auto')
            #vmin=0, vmax=15, origin='lowest', aspect='auto')
    plt.xlabel(gc_axis_label, fontsize=fontsize); plt.ylabel(temp_axis_label , fontsize=fontsize)
    plt.xticks(fontsize=fontsize); plt.yticks(fontsize=fontsize)
    plt.suptitle(power_axis_label,fontsize=fontsize)
    plt.colorbar(label= power_axis_label)
    plt.savefig(folder_address + '\\' + power_axis_label + '.png', bbox_inches='tight')
    colormap_figures.append(power_axis_label + '.png')
    plt.show()
    
    plt.figure(figsize = (10,8))
    plt.imshow(frequency_temp_vhflipped, extent=[min(gc), max(gc), min(temp), max(temp)], cmap='RdBu',
                origin='lowest', aspect='auto')
               #vmin=194.18, vmax=194.25, origin='lowest', aspect='auto')
    plt.xlabel(gc_axis_label, fontsize=fontsize); plt.ylabel(temp_axis_label , fontsize=fontsize)
    plt.xticks(fontsize=fontsize); plt.yticks(fontsize=fontsize)
    plt.suptitle(frequency_axis_label,fontsize=fontsize)
    plt.colorbar(label= frequency_axis_label)
    plt.savefig(folder_address + '\\' + frequency_axis_label + '.png', bbox_inches='tight')
    colormap_figures.append(frequency_axis_label + '.png')
    plt.show()
    
    if file_name_powerpoint != '' :
        images_to_powerpoint(file_name_powerpoint, [temp_axis_label + '_sweep.png'], slide_title = 'Temperature sweep', slide_label = data_title)
        images_to_powerpoint(file_name_powerpoint, [gc_axis_label + '_sweep.png'], slide_title = 'GC sweep', slide_label = data_title)
        #images_to_powerpoint(file_name_powerpoint, colormap_figures, slide_title = 'GC sweep')

        images_to_powerpoint(file_name_powerpoint, colormap_figures, slide_title = '2D Plotting', slide_label = data_title)
    
    return ax

# FSCS angle tuning analysis

In [ ]:
def analyze_crystal_tuning(
     file_name='', file_labels = [] , tuning='angle' # or 'temperature'
):
    
    temperatures = {}
    angles = {}; angle_values = []
    peak_rightmost = []; peak_leftmost = []
    dip_rightmost = [] ;  dip_leftmost = []


    data_title = file_name
    data_pd, data, data_labels = import_data(folder_address='Inputs', file_name= file_name, file_labels = file_labels, header_rows=1, concatenate=0)
    x_axis_label = {}; y_axis_label = {}; x={} ; y = {}; peaks={}; dips={} 

    figsize = (16,12); fontsize = 24
    fig, ax = plt.subplots(figsize=figsize)
    plt.rcParams['axes.linewidth'] = 3 #set the value globally
    line_kwargs = {'ls' :'--', 'lw' : 2, 'ms' : 12}

    #for file_label in ['stage_4-minus20_degrees_2019_02_14']:
    for file_label in file_labels:

        # auto-detect the angles and temperatures from the file_label
        
        if  re.findall('(\d+)C', file_label) :# works on integer temperatures only
            temperature = re.findall('(?:(\w+.\d+)C|(\w+)C)' , file_label )[0][0] + re.findall('(?:-(\d+.\d+)C|-(\d+)C)' , file_label )[0][1]
            
            if 'minus' in temperature:
                temperature = temperature.replace('minus','-')
            temperature = float(temperature)
            temperatures[file_label] = temperature

        
        #angle = re.findall('-(.*?)_degrees', file_label)[0]
        angle = re.findall('-(\w+)_degrees', file_label)[0]
      
        if 'minus' in angle:
            angle = angle.replace('minus','-')
            
        angle = float(angle)
        angles[file_label] = angle
        angle_values.append(angle)

        x_axis_label[file_label]=data_labels[file_label][0]
        y_axis_label[file_label]=data_labels[file_label][1]
        x[file_label] = data_pd[file_label] [ x_axis_label[file_label] ].values
        y[file_label] = data_pd[file_label] [ y_axis_label[file_label] ].values    

        peaks[file_label], _ = find_peaks( y[file_label], distance=5000, prominence= 0.15)
        dips[file_label] , _ = find_peaks(-y[file_label], distance=5000, prominence= 0.15)

        ax.plot(x[file_label][peaks[file_label]], y[file_label][peaks[file_label]], 'x',label='', markeredgewidth=5, markersize=12)
        ax.plot(x[file_label][dips [file_label]], y[file_label][dips [file_label]], 'x',label='', markeredgewidth=5, markersize=12)


        for i, txt in enumerate( x[file_label][peaks[file_label]]):
            ax.annotate('%.2f' %txt, (x[file_label][peaks[file_label]][i], y[file_label][peaks[file_label]][i]), xytext = (-20, -45), textcoords='offset pixels', arrowprops={'arrowstyle': '->'} )
        for i, txt in enumerate( x[file_label][dips [file_label]]):
            ax.annotate('%.2f' %txt, (x[file_label][dips [file_label]][i], y[file_label][dips [file_label]][i]), xytext = (-20, -45), textcoords='offset pixels', arrowprops={'arrowstyle': '->'} )
    
        ax.plot(x[file_label],y[file_label], label=angles[file_label], **line_kwargs)
        if x[file_label][peaks[file_label]][-1] < 1564.5:
            peak_rightmost.append(x[file_label][peaks[file_label]][-1])
        else : 
            peak_rightmost.append(x[file_label][peaks[file_label]][-2])

        peak_leftmost.append(x[file_label][peaks[file_label]][0])
        dip_leftmost.append( x[file_label][ dips[file_label]][0])
        dip_rightmost.append( x[file_label][ dips[file_label]][0])


    ax.set_xlabel(x_axis_label[file_label], fontsize = fontsize)
    ax.set_ylabel(y_axis_label[file_label], fontsize = fontsize)    
    grid_kwargs = {'color':'gray', 'ls': '--', 'lw': 1}
    ax.grid(b=None, which='both', axis='both', **grid_kwargs)
    ax.tick_params(direction='in', labelsize=fontsize)
    ax.legend(loc='best', fontsize = fontsize, title='Angle (°)')

    plt.show()
    
    figsize = (12,6); fontsize = 24
    fig = plt.figure(1, figsize = figsize)

    ax=plt.subplot(221)
    line_kwargs['marker'] = 'o'; line_kwargs['markerfacecolor'] = 'None'; line_kwargs['markeredgewidth'] = 2
    ax.plot(angle_values, peak_leftmost, label=file_label, **line_kwargs)
    grid_kwargs = {'color':'gray', 'ls': '--', 'lw': 1}
    ax.grid(b=None, which='both', axis='both', **grid_kwargs)
    ax.tick_params(direction='in', labelsize=fontsize/1.5)
    ax.set_title('Leftmost Peak' , fontsize = fontsize); ax.set_xlabel("Angle (°)", fontsize = fontsize) ; ax.set_ylabel("λ (nm)", fontsize = fontsize)    

    ax=plt.subplot(222)
    ax.plot(angle_values, peak_rightmost, label=file_label, **line_kwargs, color='r')
    grid_kwargs = {'color':'gray', 'ls': '--', 'lw': 1}
    ax.grid(b=None, which='both', axis='both', **grid_kwargs)
    ax.tick_params(direction='in', labelsize=fontsize/1.5)
    ax.set_title('Rightmost Peak' , fontsize = fontsize); ax.set_xlabel("Angle (°)", fontsize = fontsize) ; ax.set_ylabel("λ (nm)", fontsize = fontsize)    

    ax=plt.subplot(223)
    line_kwargs['marker'] = 'o'; line_kwargs['markerfacecolor'] = 'None'; line_kwargs['markeredgewidth'] = 2
    ax.plot(angle_values, dip_leftmost, label=file_label, **line_kwargs)
    grid_kwargs = {'color':'gray', 'ls': '--', 'lw': 1}
    ax.grid(b=None, which='both', axis='both', **grid_kwargs)
    ax.tick_params(direction='in', labelsize=fontsize/1.5)
    ax.set_title('Leftmost Dip' , fontsize = fontsize); ax.set_xlabel("Angle (°)", fontsize = fontsize) ; ax.set_ylabel("λ (nm)", fontsize = fontsize)    

    ax=plt.subplot(224)
    ax.plot(angle_values, dip_rightmost, label=file_label, **line_kwargs, color='r')
    grid_kwargs = {'color':'gray', 'ls': '--', 'lw': 1}
    ax.grid(b=None, which='both', axis='both', **grid_kwargs)
    ax.tick_params(direction='in', labelsize=fontsize/1.5)
    ax.set_title('Rightmost Dip' , fontsize = fontsize); ax.set_xlabel("Angle (°)", fontsize = fontsize) ; ax.set_ylabel("λ (nm)", fontsize = fontsize)    

    
    plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=0.95, hspace=0.55, wspace=0.45)
    plt.show()
    
    return peaks, angles, temperatures

# Interactive Functions

In [7]:
def get_pid(heater ='bench2'):

    return [ float(pid) for pid in re.findall('[0-9-.]+' , board_io('%s pid' %heater)[-1]) ] 

def gain_current(gc_write=-1):
    
    if gc_write == -1:
        gc_read = board_io( 'gc')[-1]
        return re.findall('[0-9-.]+' , gc_read)[0] # extract the current from the returned string
    
    else:
        gc_read = board_io( 'gc %s' %gc_write)[-1]
        return re.findall('[0-9-.]+' , gc_read)[0] # extract the current from the returned string

def mzi(mzi_write, n=0):
    
    if n not in [1,2,3,4]:
        raise ValueError('mzi%s is not a valid mzi' %n)
    mzi_read = board_io( 'tps pp mzi%d %s' %(n, mzi_write))[-1]
    return re.findall('[0-9-.]+' , mzi_read)[-1] # extract the mzi from the returned string
    
    
try:
    # Slider for Gain Current 
    # Initialize with the Current value to not change it
    gc_current = round(float (re.findall('[0-9-.]+' , board_io('gc')[-1] )[0] ), 1)
    gain_current_slider = interactive( gain_current, gc_write=widgets.FloatSlider(min=0,max=400,step=1,value=gc_current, description='Gain Current (mA)' , style=style) )

except:
    pass

try:
    # Sliders for MZI1
    # Initialize with the Current value to not change it
    mzi1 = float (re.findall('[0-9-.]+' , board_io('tps pp mzi1')[-1] )[-1] )
    mzi1_slider = interactive( mzi, n=fixed(1) , mzi_write=widgets.FloatSlider(min=-40,max=40,step=0.5,value = mzi1, description='MZI 1 (mW)' , style=style) )

    mzi2 = float (re.findall('[0-9-.]+' , board_io('tps pp mzi2')[-1] )[-1] )
    mzi2_slider = interactive( mzi, n=fixed(2), mzi_write=widgets.FloatSlider(min=-40,max=40,step=0.5,value = mzi2, description='MZI 2 (mW)' , style=style) )

    mzi3 = float (re.findall('[0-9-.]+' , board_io('tps pp mzi3')[-1] )[-1] )
    mzi3_slider = interactive( mzi, n=fixed(3), mzi_write=widgets.FloatSlider(min=-40,max=40,step=0.5,value = mzi3, description='MZI 3 (mW)' , style=style) )

    mzi4 = float (re.findall('[0-9-.]+' , board_io('tps pp mzi4')[-1] )[-1] )
    mzi4_slider = interactive( mzi, n=fixed(4), mzi_write=widgets.FloatSlider(min=-40,max=40,step=0.5,value = mzi4, description='MZI 4 (mW)' , style=style) )

except:
    pass


def heater_temp(x, heater):
    board_io( '%s target %s' %(heater,x))
    
def set_bench_mode(mode):
    print(mode)
    mode = {'Off': 'off', 'Temperature': 'temp', 'OOP':'oop' }[mode]
    board_io( 'bench2 mode %s' %mode)

def set_bench_scale(scale, phi, mod_frequency, mod_gain, mod_averaging):
    board_io( 'bench2 scale %s' %scale)
    board_io( 'dac phi %s' %phi)
    board_io('mod enable %s %s %s' %(mod_frequency, mod_gain, mod_averaging))
    
    
def enable_heater(state, heater):
    board_io( '%s enable %s' %(heater,state))

def DUT(x):
    print('%s is selected' %x)
    return x
try:
    DUT_current = DUT_selector.result
except:
    DUT_current = 'DUT9'
DUT_selector = interactive(DUT, x = widgets.Dropdown( options = ['DUT8', 'DUT9', 'DUT10','DUT13'], value = DUT_current,  description='DUT:'))

NameError: name 'interactive' is not defined

# Initialize Smart Board

In [9]:
def initialize_dut(DUT):
    

    # Initialize DUT, import parameters
    DUT_label = '%s initialization' %DUT
    data_pd, data, data_labels = import_data(folder_address='Inputs', file_labels=[DUT_label], file_name= '', header_rows=2, numeric=0, concatenate=0)
    
    DUT_parameters = [re.findall( '(.*)\_\*', data_label)[0] for data_label in data_labels[DUT_label]]
    DUT_parameter_commands = [re.findall( '\*(.*)\*', data_label)[0] for data_label in data_labels[DUT_label]]
    
    for (command,value) in zip(DUT_parameter_commands, data[DUT_label][0] ):
        print ('%s: %s' %(command, value))
        board_io(command +' %s' %value)        
    
    print(data_labels[DUT_label])
    
#     k_benchDUT = [float(data_pd[file_label]['k_bench_p'].iloc[0,0]), float(data_pd[file_label]['k_bench_i'].iloc[0,0]), float(data_pd[file_label]['k_bench_d'].iloc[0,0])]
#     k_comb_DUT = [float(data_pd[file_label]['k_comb_p'].iloc[0,0]), float(data_pd[file_label]['k_comb_i'].iloc[0,0]), float(data_pd[file_label]['k_comb_d'].iloc[0,0])]
#     k_fscs_DUT = [float(data_pd[file_label]['k_fscs_p'].iloc[0,0]), float(data_pd[file_label]['k_fscs_i'].iloc[0,0]), float(data_pd[file_label]['k_fscs_d'].iloc[0,0])]

#     # set the default bench, COM5, FSCS loop pid values
#     board_io( 'bench2 pid %s %s %s' %(tuple(k_benchDUT)))
#     board_io( 'comb pid %s %s %s' %(tuple(k_comb_DUT)))
#     board_io( 'fscs pid %s %s %s' %(tuple(k_fscs_DUT)))

    # Enable ac modulation
    board_io( 'dac phi 2')
    board_io( 'bench2 scale -0.1')
    board_io( 'mod enable 1000 4 255')
    
    return data_pd, data_labels

In [10]:
def save_dut_parameters(DUT):
    print('This function will work after issue #48 on GitHub is resolved, other corrections are also needed')
    
    # Initialize DUT, import parameters
    DUT_label = '%s initialization' %DUT
    data_pd, data, data_labels = import_data(folder_address='Inputs', file_labels=[DUT_label], file_name= '', header_rows=2, numeric=0, concatenate=0)
    
    DUT_parameters = [re.findall( '(.*)\_\*', data_label)[0] for data_label in data_labels[DUT_label] ]
    DUT_parameter_commands = [re.findall( '\*(.*)\*', data_label)[0] for data_label in data_labels[DUT_label] ]
    values = []
    
    for command in DUT_parameter_commands:
        
        value = re.findall('[0-9-.]+' , board_io(command)[-1] )[0]
        #value = board_io(command)[-1]
        print ('%s: %s' %(command, value))
        values.append(value)
    
    DUT_parameter_commands_to_export = ['*' + DUT_parameter_command + '*' for DUT_parameter_command in DUT_parameter_commands]
    index = pd.MultiIndex.from_arrays([DUT_parameters, DUT_parameter_commands_to_export])
    values_pd = pd.DataFrame([values], columns=index)
    
    #print (index)
    #print(values)
    export_pd( values_pd, file_name=DUT_label, index=False, header=True)


# Terminate connection with the SmartBoard

In [1]:
def disconnect_smartboard():
    
    try:

        # Turn off the GC
        gc_read = board_io( 'gc 0')[-1]
        gc_read = float( re.findall('[0-9-.]+' , gc_read)[0] ) # extract the current from the returned string
        print('Gain Current is %.3f mA' %gc_read)

        # Disable the bench
        board_io( 'bench2 mode temp') # go to the thermal control mode first
        bench_disabled = board_io( 'bench2 enable off')
        print('bench2 TEC is disabled')

        # Disable the comb heater
        comb_heater_disabled = board_io( 'comb enable off')
        board_io( 'comb heat 0')
        print('Comb heater is disabled')

        # Disable the FSCS heater
        fscs_heater_disabled = board_io( 'fscs enable off')
        board_io( 'fscs heat 0' )
        print('FSCS heater is disabled')

        # Disable the modulation
        board_io( 'mod disable')

        smartboard.close()
        print('SmartBoard is disconnected')
    
    except:
        print('Is Smartboard connected?')
    



interact_shutdown = interact_manual.options(manual_name="Disconnect Smart Board"); 
disconnect_board_button = interact_shutdown(disconnect_smartboard)

NameError: name 'interact_manual' is not defined